# Part 5 Mapping and variant calling

## 5.1 Read mapping

### Exercise 5.1.1 Comparing Capsella rubella and Capsella grandiflora

1. Align the two sequences to find conserved and divergent regions.

2. Whole genome shotgun sequencing was performed using the Monte Gargano (Italy) reference strain. Reads were collected with Sanger, Roche 454 XLR and Illumina GAIIx machines. Assembly was performed with Arachne v.20071016. Annotation was performed using tophat 1.3.0 and cufflinks 1.0.3.

3. Busco analysis of the assembly shows Single copy genes were present 98.3%, suggesting a high quality assembly.

4. On NIH Genbank, accessions from the article: Entrez BioProject ID PRJNA13878

### Exercise 5.1.2 Read mapping

1. Read mapping is aligning sequence reads to a reference genome. Blast takes one sequence and compares it to another sequence or a database, while read mapping takes millions of short reads (or long reads), and aligns them to a reference genome. Blast algorithm would be too slow to align the millions of reads.
2.
- FreeBayes
- Jitterbug
- Fermi-Kit
- CommonLaw
- MetaSV

### Exercise 5.1.3 Capsella rubella reference genome

1. bwa installed in /programs

2. gff3,rna,cds,protein,genome,seq-report.
- gff3 describes the locations and types of genomic features (genes, mRNAs, exons, CDS, UTRs, repeats, etc.) on a reference genome.
- rna denotes a FASTA file containing the full transcript sequences (mRNA)
- CDS (Coding DNA Sequences): a FASTA file containing only the coding regions of each transcript
- Protein means a FASTA file with the amino acid sequences translated from the CDS sequences for each predicted protein coding gene.
- Genome is the FASTA file of the reference genome assembly itself
  
3. Genome FASTA and gff3

### Exercise 5.1.4 Read mapping against the reference genome

1. ```
    wget https://github.com/samtools/samtools/releases/download/1.22.1/samtools-1.22.1.tar.bz2
    tar xvfj samtools-1.22.1.tar.bz2
    cd samtools-1.22.1/
    make
   ```
2. ```~/p5/010-run-bwa.sh```
3. Output files are sam files. Sam files stand for Sequence Alignment/Map files.
   The first part has metadata lines starting with @. @HD refers to file format/version info. @SQ refers to reads sequence names and lengths. @PG can refer to programs used to generate alignments.
   The next segment contains the alignment information. It is one line per read. This one starts with the line:
   ```
   N73019:1:1:1103:6775#0  99      NW_006238923.1  1058971 60      120M    =       1059205 354     GAAGGTATGACTCTACACGTTTCATACCAATCTTTAGACTTTCTTCTGTGGTCTGCATTCTCAATCCTGGTACCTCTTTCCAAGGAATGTTGAATTTTTCTTTTCATATGTTATTTCCTA CCCACABDDCDAE=EFDFDFCDE=EBC?EEEFFFFEDFE??=?CCBEE@5EBCB?BCE5CEEA:FAD5A?5?A?>C@>>C>>->>@BBBB:AB9BA:B?@;-'5/5>>=>@@:B@EB5@@ NM:i:1  MD:Z:102C17     MC:Z:120M MQ:i:60 AS:i:115        XS:i:0
    ```
N73019:1:1:1103:6775#0 is the name of the read
99 is the bitwise flag
NW_006238923.1 is the reference sequence name this read maps to
1058971 is the leftmost position of the alignment
60 mapping quality
120M is the CIGAR string, here means 120 match
= RNEXT, The next read alignment, whether it is to the same reference or not, in paired end the next one is the mate, and here it maps to the same reference
1059205 position of the mate read on the reference
354 template length (distance between outer ends of the pair reads)
Then the sequence of the read
CCCACABDDCDAE=EFDFDFCDE=EBC?... Base quality scores in ASCII
rest are optional tags

4. RNAME gives the reference genome or which chromosome the read mapped to, and POS gives the exact position of the mapping.

### Exercise 5.1.5 Explore the alignments

1. Samtools is used for Reading/writing/editing/indexing/viewing SAM/BAM/CRAM format.
2. installed under ~/programs
3. BAM files are SAM files in binary form.
4. https://www.htslib.org/doc/1.13/samtools-view.html, p5/020-sam2bam.sh, p5/021-samtools-index.sh
5. p5/023-samtools-count.sh
6. ```samtools flagstat bam_files/Cr1504.sorted.bam```
7. 024-samtools-filter.sh

### Exercise 5.1.6 Visual inspection
1. Samtools tview is an alignment viewing tool.
2. samtools tview your_sorted.bam your_reference.fasta
   To navigate in tview:
    - Arrow keys: Move left/right (one column at a time) or up/down (one row at a time).
    - g: Go to a specific genomic position (you will be prompted to enter the position).
    - q: Quit tview.
    - N: Switch to the next viewing mode.
    - ?: Display help with more commands.
   fasta files, Gene Annotation Files:GTF (.gtf) and GFF (gff), Alignment Files (bam), variant files(vcf) and others

### Exercise 5.1.8 Comparing raw and pre-processed reads
1. p5/030-mapping-corrected

### Exercise 5.1.11 Mapping whole genome data
1. ~/p5/040-mapping-bowtie.sh

### Exercise 5.1.12 Data from different technologies
1. Canu, Falcon, minimap/miniasm were the assemblers used.
2. minimap2 -a ref.fa query.fq > alignment.sam from https://github.com/lh3/minimap2
   
4. Illumina MiSeq: High accuracy, short reads
PacBio: Long reads, higher error rates
Oxford Nanopore: Long reads, variable error rates

## 5.2 Genomic variants

### Exercise 5.2.1 Variant calling
1. Variant calling analyzes sequencing data to detect true genetic changes while distinguishing them from sequencing errors or artifacts.
   Types:
   Single nucleotide polymorphism/variant (SNP/SNV): change of a single base in the DNA sequence (1 base)
   Insertion/deletion: addition of a group of bases, can cause frame shift (1-20 bases)
   Structural variants: deletions, duplications, inversions, translocations, >20 bases
   Copy number variations: gain or loss of large DNA segments, affecting gene dosage (hundreds to millions of bases)
   Multi-nucleotide Variant (MNV): multiple adjacent base changes that are inherited together, (contiguous 2+ bases)
   Complex/Other Variants: 	Includes mobile element insertions, gene conversions, and more
2. bcftools mpileup -f reference.fa alignments.bam | bcftools call -mv -Ob -o calls.bcf
   p5/050-variant-calling-samtools.sh
3. the output is in vcf format. The first header line denotes the file format, and the second header line is as follows:
   #CHROM  POS     ID      REF     ALT     QUAL    FILTER  INFO    FORMAT  /mnt/silo/hts2024/dnarabaci/p5/bam_files/Cg926.sorted.bam       /mnt/silo/hts2024/dnarabaci/p5/bam_files/Cr1504.sorted.bam
4. IGV
5. Script p5/051... to generate hom_snps, then the following command gives 2855: grep -v '^#' hom_snps.vcf | wc -l 
   


### Exercise 5.2.2 Comparing variant callers

1. Clair3, DeepVariant, Octopus, GATK, FreeBayes, and Strelka2 (From [Barbitoff et al, 2022](https://rdcu.be/eNuwr))
2. In the article, they compared variant calling tools, and DeepVariant consistently performed the best with high robustness.